### 1. Functions and variables for collecting predictions

In [ ]:
import os
import numpy as np
import cv2
from detection import detect_face_for_testing
from feature_extraction import predict_spoof
from anti_spoof import load_antispoof_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

DATASET_PATH_REAL = "LCC_eval_pics/real"
DATASET_PATH_ATTACK = "LCC_eval_pics/spoof"
SAVE_PATH = 'results_FAS/'
N_REAL = 314
N_SPOOFED = 700
SPOOF_THRESHOLDS = np.concatenate(([0], np.arange(0.94, 1.00, 0.01)))

antispoof_sess, antispoof_input = load_antispoof_model()

def get_image_paths(dataset_path, n_images):
    # Get n random image paths from the dataset
    image_paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(('.jpg', '.png'))]
    sampled_image_paths = np.random.choice(image_paths, n_images, replace=False)

    return sampled_image_paths

def get_predictions(dataset_path, correct_pred, thresholds, n_pred):
    n_undetected = 0
    y_pred_li = [[] for _ in range(len(thresholds))] # Initialize a list of predictions for each threshold
    
    image_paths = get_image_paths(dataset_path, n_pred)

    for image_path in image_paths: 
        image_array = cv2.imread(image_path)

        if image_array is None:
            print(f"Error reading image: {image_path}")
            continue
        
        face_data, image_rgb = detect_face_for_testing(image_array)

        if face_data is not None:

            for i, threshold in enumerate(thresholds):
                # Check if the image is spoofed based on the face data
                pred = predict_spoof(face_data, image_rgb, antispoof_sess, antispoof_input, threshold)

                if pred != correct_pred:
                    #print(f'Incorrect prediction at: {image_path}')
                    pass

                y_pred_li[i].append(pred) # Append the prediction to the list for the current threshold
        else:
            n_undetected += 1
            #print(f"No face detected in image: {image_path}")
        
    print(f"Number of undetected images: {n_undetected}")
    return y_pred_li, n_undetected

def run_test(path_real, path_attack, thresholds, n_real, n_spoofed, test_n):
    # Collect one list of predictions for non-spoofed images for every threshold
    y_pred_real_li, n_undetected_real = get_predictions(path_real, True, thresholds, n_real)
    y_true_real = np.full(len(y_pred_real_li[0]), True)
    print(f"Number of real images: {len(y_pred_real_li[0])}")

    # Collect one list of predictions for spoofed images for every threshold
    y_pred_attack_li, n_undetected_attack = get_predictions(path_attack, False, thresholds, n_spoofed)
    y_true_attack = np.full(len(y_pred_attack_li[0]), False)
    print(f"Number of spoofed images: {len(y_pred_attack_li[0])}")

    y_pred_li = [y_pred_real + y_pred_attack for (y_pred_real, y_pred_attack) in zip(y_pred_real_li, y_pred_attack_li)]
    y_true = np.concatenate((y_true_real, y_true_attack))

    rows = []

    for y_pred, threshold in zip(y_pred_li, thresholds):
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        rows.append({
            'Test Number': test_n,
            'Threshold': threshold,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-score': f1,
            'Undetected Real Images': n_undetected_real,
            'Undetected Spoofed Images': n_undetected_attack
        })

        print(f"Threshold: {threshold:.2f}")
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}\n")

    df = pd.DataFrame(rows)

    return y_pred_li, y_true, df



### 2. Collecting predictions on images for each threshold, then plotting ROC curves and confusion matrices

In [ ]:
from plotting import plot_conf_mat, plot_ROC, get_tpr_and_fpr

test_n_li = np.arange(1, 11) # Number of tests to run

total_test_data = pd.DataFrame()

tpr_li_li = []
fpr_li_li = []

for test_n in test_n_li:
    print(f"Test number: {test_n}")

    # Get a list of predictions and performance data for each threshold
    y_pred_li, y_true, test_data = run_test(DATASET_PATH_REAL, DATASET_PATH_ATTACK, SPOOF_THRESHOLDS, N_REAL, N_SPOOFED, test_n)
    total_test_data = pd.concat([total_test_data, test_data], ignore_index=True)

    # Visualize the results using ROC curve
    plot_ROC(y_true, y_pred_li, SPOOF_THRESHOLDS, test_n)

    # Get TPR and FPR for each test to later compute average ROC curve
    tpr_li = []
    fpr_li = []

    for y_pred in y_pred_li:
        tpr, fpr = get_tpr_and_fpr(y_true, y_pred)
        tpr_li.append(tpr)
        fpr_li.append(fpr)

    tpr_li_li.append(tpr_li)
    fpr_li_li.append(fpr_li)
    
    # Visualize the results for each threshold using confusion matrices
    for i, y_pred in enumerate(y_pred_li):
        print(f"Confusion matrix for threshold {SPOOF_THRESHOLDS[i]:.2f}:")
        plot_conf_mat(y_true, y_pred, SPOOF_THRESHOLDS[i], test_n)


#### Plotting the average ROC curve across all tests

In [ ]:
def get_average_rates(rate_li_li):
    rates_df = pd.DataFrame(rate_li_li)
    average_rates = list(rates_df.mean(axis=0))

    return average_rates

avg_tpr_li = get_average_rates(tpr_li_li)
avg_fpr_li = get_average_rates(fpr_li_li)

# Visualize the average ROC curve
plot_ROC(y_true, y_pred_li, SPOOF_THRESHOLDS, 'AVG', avg_rates=(avg_tpr_li, avg_fpr_li))


### 3. Analyze and save the results

In [ ]:
# Get the average performance metrics for each threshold across all tests
total_test_data = total_test_data.drop(columns=["Test Number"])
average_metrics = total_test_data.groupby("Threshold").mean(numeric_only=True).reset_index()

# Inspect the average metrics
print("Average metrics across all tests:")
average_metrics

In [ ]:
# Save the results to CSV and Excel files
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

total_test_data.to_csv(f"{SAVE_PATH}test_results_FAS.csv", index=False)
total_test_data.to_excel(f"{SAVE_PATH}test_results_FAS.xlsx", index=False)

average_metrics.to_csv(f"{SAVE_PATH}average_metrics_FAS.csv", index=False)
average_metrics.to_excel(f"{SAVE_PATH}average_metrics_FAS.xlsx", index=False)